# Plot GHRSST night time scenes

Use data on IMOS s3 bucket

http://imos-data.s3-website-ap-southeast-2.amazonaws.com/?prefix=IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/

In [1]:
import xarray as xr
import s3fs

import cmocean # Oceanography colour maps
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from utils import plot_swot_basemap

In [3]:
plt.rcParams['font.size']=14
plt.rcParams['axes.labelsize']='medium'

In [4]:
# Login to the s3 bucket and list the folders in the base directory of the imos-data/ bucket

fs = s3fs.S3FileSystem(anon=True) # Note that we can login anonymously because this bucket allows it

aodn = fs.ls('imos-data/')
for item in aodn:
    print(item)

imos-data/AIMS
imos-data/Bureau_of_Meteorology
imos-data/CSIRO
imos-data/Deakin_University
imos-data/Defence_Technology_Agency-New_Zealand
imos-data/Department_of_Defence
imos-data/Department_of_Environment_and_Science-Queensland
imos-data/Department_of_Planning_and_Environment-New_South_Wales
imos-data/Department_of_Transport-Western_Australia
imos-data/Derwent_Estuary_Program
imos-data/Flinders_University
imos-data/Future_Reef_MAP
imos-data/Gippsland-Ports-Victoria
imos-data/IMOS
imos-data/Macquarie_University
imos-data/NSW-OEH
imos-data/Pilbara_Ports_Authority
imos-data/Southern_Cross_University
imos-data/TERN
imos-data/UNSW
imos-data/UON
imos-data/UWA
imos-data/WAMSI
imos-data/aodn.js
imos-data/error.html
imos-data/index.html
imos-data/robots.txt


In [5]:
timestr ='20230[456]'
aodn = fs.glob('imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/{}*.nc'.format(timestr))
for item in aodn:
    print(item)



imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230401152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230402152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230403152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230404152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230405152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230406152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230407152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230408152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_night.nc
imos-data/IMOS/SRS/SST/ghrsst/L3C-1d/ngt/snpp/2023/20230409152000-ABOM-L3C_GHRSST-SSTskin-VIIRS_NPP-1d_n

In [ ]:
def plot_scene(ncfile, timestr):
    print(timestr)
    # Load the data set using xarray and s3fs
    fileobj = fs.open(ncfile)
    sst = xr.open_dataset(fileobj)
    x1,x2=121,125
    y1,y2=-15.5,-12
    
    mysst = sst.sel(lon=slice(x1,x2), lat=slice(y2,y1))
    
    dx, dy = np.gradient(mysst['sea_surface_temperature'].squeeze())
    x,y = mysst.lon, mysst.lat

    plt.figure()
    ax=plt.subplot(111)
    mysst['sea_surface_temperature'].plot(cmap='cmo.thermal',vmin=26+273, vmax=29+273,cbar_kwargs={"shrink": 0.3,})
    
    plt.pcolormesh(x,y,np.abs(dx+1j*dy), vmin=0, vmax=0.1, cmap='bone_r', alpha=0.5)
    c=plot_swot_basemap(ax, (x1,x2), (y1,y2))
    plt.tight_layout()
    plt.savefig('../FIGURES/GHRSST_1km_{}.png'.format(timestr), dpi=150)

In [ ]:
for ncfile in aodn:
    timestr=ncfile.split('/')[-1][0:8]
    print(timestr)
    plot_scene(ncfile, timestr)

In [ ]:
mysst['sea_surface_temperature'].values.shape